In [2]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [3]:
data_dir = 'https://mirror.coggle.club/dataset/coggle-competition/'
train_data = pd.read_csv(data_dir + 'intent-classify/train.csv', sep='\t', header=None)
test_data = pd.read_csv(data_dir + 'intent-classify/test.csv', sep='\t', header=None)

In [4]:
train_data["input"] = ""
train_data.columns = ["instruction", "output", "input"]

In [5]:
ds = Dataset.from_pandas(train_data)
ds[:3]

{'instruction': ['还有双鸭山到淮阴的汽车票吗13号的', '从这里怎么回家', '随便播放一首专辑阁楼里的佛里的歌'],
 'output': ['Travel-Query', 'Travel-Query', 'Music-Play'],
 'input': ['', '', '']}

In [6]:
tokenizer = AutoTokenizer.from_pretrained("/home/lyz/hf-models/Qwen/Qwen1.5-1.8B-Chat/", use_fast=False, trust_remote_code=True)
tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2Tokenizer(name_or_path='/home/lyz/hf-models/Qwen/Qwen1.5-1.8B-Chat/', vocab_size=151643, model_max_length=32768, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n现在进行意图分类任务<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [8]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)

Map:   0%|          | 0/12100 [00:00<?, ? examples/s]

In [9]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n现在进行意图分类任务<|im_end|>\n<|im_start|>user\n还有双鸭山到淮阴的汽车票吗13号的<|im_end|>\n<|im_start|>assistant\nTravel-Query<|endoftext|>'

In [10]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'Travel-Query<|endoftext|>'

In [11]:
train_ds = Dataset.from_pandas(train_data.iloc[:-2000])
eval_ds = Dataset.from_pandas(train_data[-2000:])

train_tokenized_id = train_ds.map(process_func, remove_columns=ds.column_names)
eval_tokenized_id = eval_ds.map(process_func, remove_columns=ds.column_names)

Map:   0%|          | 0/10100 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [12]:
import torch

model = AutoModelForCausalLM.from_pretrained("/home/lyz/hf-models/Qwen/Qwen1.5-1.8B-Chat/", device_map="auto")
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Line

In [13]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [14]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'v_proj', 'o_proj', 'down_proj', 'k_proj', 'gate_proj', 'up_proj', 'q_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [15]:
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 7,495,680 || all params: 1,844,324,352 || trainable%: 0.4064187512284173


In [16]:
args = TrainingArguments(
    output_dir="./output_Qwen1.5",
    per_device_train_batch_size=6,
    gradient_accumulation_steps=4,
    logging_steps=100,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=50,
    num_train_epochs=5,
    save_steps=50,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    load_best_model_at_end=True
)

In [17]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tokenized_id,
    eval_dataset=eval_tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [18]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/lyz/anaconda3/envs/py311/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
50,No log,0.296269
100,0.347000,0.247612
150,0.347000,0.229677
200,0.044600,0.226091
250,0.044600,0.196769
300,0.035700,0.201283
350,0.035700,0.198062
400,0.034600,0.181151
450,0.034600,0.208037


/home/lyz/anaconda3/envs/py311/lib/python3.11/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /home/lyz/hf-models/Qwen/Qwen1.5-1.8B-Chat/ - will assume that the vocabulary was not modified.
  warnings.warn(
/home/lyz/anaconda3/envs/py311/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/lyz/anaconda3/envs/py311/lib/python3.11/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /home/lyz/hf-models/Qwen/Qwen1.5-1.8B-Chat/ - will assume that the vocabulary was not modified.
  warnings.warn(
/home/lyz/anaconda3/env

KeyboardInterrupt: 

In [20]:
%%time

prompt = '''去北京如何走？'''
messages = [
    {"role": "system", "content": "现在进行意图分类任务"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
device = 'cuda'
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512,
    
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

CPU times: user 216 ms, sys: 44.7 ms, total: 260 ms
Wall time: 258 ms


'Travel-Query'

In [21]:
from tqdm import tqdm_notebook
pred_label = []
for train_text in tqdm_notebook(test_data[0].values):
    prompt = f'''{train_text}'''
    messages = [
    {"role": "system", "content": "现在进行意图分类任务"},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512,
        
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    pred_label += [response]

/tmp/ipykernel_34703/1740621863.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for train_text in tqdm_notebook(test_data[0].values):


  0%|          | 0/3000 [00:00<?, ?it/s]

In [22]:
pd.DataFrame({
    'ID': range(1, len(pred_label) + 1),
    'Target': pred_label,
}).to_csv('nlp_submit.csv', index=None)

# 提交一下吧~

In [23]:
pred_label[:20]

['TVProgram-Play',
 'HomeAppliance-Control',
 'Audio-Play',
 'Alarm-Update',
 'HomeAppliance-Control',
 'FilmTele-Play',
 'FilmTele-Play',
 'Music-Play',
 'Calendar-Query',
 'Video-Play',
 'Alarm-Update',
 'Music-Play',
 'Travel-Query',
 'Video-Play',
 'Radio-Listen',
 'Alarm-Update',
 'Video-Play',
 'Weather-Query',
 'Weather-Query',
 'FilmTele-Play']